In [39]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import pandas as pd
import json
import numpy as np

In [40]:
X=pd.DataFrame()
Y=pd.Series()

x_train=np.array([[1,2,3,4],[1,2,3,4]])
x_test=np.array([[4,5,6,7],[4,5,6,7]])

y_train=np.array([1,2,3,4])
y_test=np.array([1,3,4,3])

/var/folders/b7/0nbwk8050kn6p836qqxnct3c0000gn/T/ipykernel_3284/812677177.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  Y=pd.Series()


In [5]:
class RegressionDataset(Dataset):

    def __init__(self,x_data,y_data) -> None:
        self.x_data=x_data
        self.y_data=y_data
    
    def __getitem__(self, index):
        return self.x_data[index],self.y_data[index]
    
    def __len__(self):
        return len(self.x_data),len(self.y_data)
    
x_train_tensor=torch.from_numpy(x_train).float()
y_train_tensor=torch.from_numpy(x_test).float()

x_test_tensor=torch.from_numpy(x_test).float()
y_test_tensor=torch.from_numpy(y_test).float()

train_data_final=RegressionDataset(x_train_tensor,y_train_tensor)
test_data_final=RegressionDataset(x_test_tensor,y_test_tensor)

In [13]:
from dataclasses import dataclass

@dataclass
class ModelInitiation:
    epochs:int=150
    batch_size_train:int=64
    batch_size_test:int=1
    lr:float=0.001
    numerical_features:int=len(X.columns)

In [14]:
train_dataloader=DataLoader(dataset=train_data_final,batch_size=ModelInitiation.batch_size_train)

test_dataloader=DataLoader(dataset=test_data_final,batch_size=ModelInitiation.batch_size_test)

In [19]:
#defining neaural networks architecture

class Regression(nn.Module):
    def __init__(self, num_features) -> None:
        super(Regression,self).__init__()

        self.layer1=nn.Linear(num_features,32)
        self.layer2= nn.Linear(32,64)
        self.layer3= nn.Linear(64,128)
        self.layer4=nn.Linear(128,256)
        self.layer_out= nn.Linear(256,1)

        self.relu= nn.ReLU()


    def forward(self,inputs):
        x1=self.relu(self.layer1(inputs))
        x2=self.relu(self.layer2(x1))
        x3=self.relu(self.layer3(x2))
        x4=self.relu(self.layer4(x3))
        x5_out=self.layer_out(x4)
        return x5_out
    
    def predict(self,test_inputs):
        x1=self.relu(self.layer1(test_inputs))
        x2=self.relu(self.layer2(x1))
        x3=self.relu(self.layer3(x2))
        x4=self.relu(self.layer4(x3))
        x5_out=self.layer_out(x4)
        return x5_out

In [20]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [24]:
model=Regression(ModelInitiation.numerical_features)
model.to(device=device)

#add the loss function and add the optimized

criterion=nn.MSELoss()
optimizers=optim.Adam(model.parameters(),lr=ModelInitiation.lr)

/opt/homebrew/lib/python3.11/site-packages/torch/nn/init.py:405: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


In [25]:
for i in model.parameters():
    print(i)

Parameter containing:
tensor([], size=(32, 0), requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)
Parameter containing:
tensor([[-0.1685, -0.0643, -0.1151,  ...,  0.0560, -0.1711,  0.0588],
        [-0.0771,  0.0433,  0.0933,  ...,  0.0943, -0.0027,  0.0179],
        [ 0.1509,  0.0537,  0.0112,  ...,  0.0377, -0.0623, -0.1110],
        ...,
        [ 0.0682,  0.1567, -0.0126,  ...,  0.0127, -0.1544,  0.1108],
        [-0.0901,  0.0437,  0.1643,  ..., -0.1622,  0.0417, -0.0633],
        [-0.0425, -0.1748, -0.0762,  ...,  0.0060,  0.0112, -0.0710]],
       requires_grad=True)
Parameter containing:
tensor([ 0.1432,  0.0358,  0.0811,  0.1567, -0.1375,  0.0114,  0.0565, -0.1005,
         0.1318,  0.0896, -0.1072, -0.0968,  0.1203, -0.0676, -0.0186,  0.1481,
        -0.1152, -0.0264,  0.1416, -0.0430, -0.0689,  0.0846,  0.1520,  0.0072,
 

In [26]:
loss_stats = {
    'train': [],
    "val": []
}


In [43]:
for e in range(1,ModelInitiation.epochs+1):
    train_epoch_loss = 0
    model.train()
    for x_train_batch,y_train_batch in train_dataloader:
        x_train_batch,y_train_batch=x_train_batch.to(device),y_train_batch.to(device)
        print(y_train_batch.unsqueeze(1))
        optimizers.zero_grad()
        y_train_pred=model(x_train_batch)
        train_loss=criterion(y_train_pred,y_train_batch.unsqueeze(1))
        train_loss.backward()
        optimizers.step()
        train_epoch_loss+=train_loss.item()
    with torch.no_grad():
        val_epoch_loss=0
        model.eval()
        for x_val_batch,y_val_batch in test_dataloader:
            y_val_pred=model(x_val_batch)
            val_loss=criterion(y_val_pred,y_val_batch.unsqueeze(1))
    loss_stats["train"].append(train_epoch_loss/len(train_dataloader))
    loss_stats["val"].append(val_epoch_loss/len(test_dataloader))
    print(f'Epoch: {e} | Train Loss: {train_epoch_loss/len(train_dataloader)} | Test Loss: {val_epoch_loss/len(test_dataloader)}')


json.dump(loss_stats,open("train_val_loss.json","w"))

In [27]:
ypred_list=[]

with torch.no_grad():
    model.eval()
    for X_batch,_ in test_dataloader:
        X_batch=X_batch.to(device)
        y_pred=model(X_batch)
        ypred_list.append(y_pred.cpu().numpy())

150

In [33]:
from sklearn.metrics import mean_squared_error,r2_score

mse=mean_squared_error(y_test,ypred_list)
r2score=r2_score(y_test,y_pred)
print(f"r2: {r2score} | mse: {mse}")